## Configuration Script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 安装最新版本Keras
# https://keras.io/
#!pip install keras
#最新版 keras==2.3.0 2020.06.27
# 指定版本安装
#!pip install keras==2.0.8
!pip install keras==2.1.5

%tensorflow_version 1.x
!pip install imgaug==0.2.5
!pip install gast==0.2.2

# 安装 OpenCV
# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
# 安装 Pytorch
# http://pytorch.org/
!pip install --upgrade pip
!pip install folium==0.2.1
!pip install numpy --upgrade
!pip install torch==1.5.1
# 安装 XGBoost
# https://github.com/dmlc/xgboost
!pip install -q xgboost
# 安装 7Zip
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
# 安装 GraphViz 和 PyDot
!apt-get -qq install -y graphviz && pip install -q pydot

#import tensorflow
#print(tensorflow.__version__)

!pip install --upgrade pip
!pip install folium==0.2.1
!pip install numpy --upgrade
!pip install torch==1.5.1


## DIODE Dataset files movement

In [ ]:
import os
import numpy as np
import shutil

basepath = "/content/drive/My Drive/Colab Notebooks/DIODE Depth"
train_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset/train"
val_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset/val"
depth_means = []
subsets = ['train', 'val']
occasions = ['indoors', 'outdoor']
for subset in subsets:
  if subset == 'train':
    destination_dir = train_dir
  elif subset == 'val':
    destination_dir = val_dir
  level1_dir = os.path.join(basepath, subset)
  for occasion in occasions:
    level2_dir = os.path.join(level1_dir, occasion)
    scenes = os.listdir(level2_dir)
    for scene in scenes:
      level3_dir = os.path.join(level2_dir, scene)
      scans = os.listdir(level3_dir)
      for scan in scans:
        data_dir = os.path.join(level3_dir, scan)
        image_files = os.listdir(data_dir)
        selected_images = [image_file for image_file in image_files if not image_file.endswith('.txt')]
        for selected_image in selected_images:
          temp_dir = os.path.join(data_dir, selected_image)
          shutil.copy(temp_dir, destination_dir)
        print('Finished moving the files in {} / {} into the destination directory'.format(scan, scene))

print("Successfully moving all the files in the dataset!")


Finished moving the files in scan_00000 / scene_00000 into the destination directory
Finished moving the files in scan_00001 / scene_00000 into the destination directory
Finished moving the files in scan_00002 / scene_00000 into the destination directory
Finished moving the files in scan_00003 / scene_00000 into the destination directory
Finished moving the files in scan_00004 / scene_00000 into the destination directory
Finished moving the files in scan_00005 / scene_00000 into the destination directory
Finished moving the files in scan_00006 / scene_00000 into the destination directory
Finished moving the files in scan_00007 / scene_00000 into the destination directory
Finished moving the files in scan_00009 / scene_00001 into the destination directory
Finished moving the files in scan_00010 / scene_00001 into the destination directory
Finished moving the files in scan_00011 / scene_00001 into the destination directory
Finished moving the files in scan_00012 / scene_00001 into the de

## DIODE Dataset Backup

In [ ]:
import os
import numpy as np
import shutil

src_train_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset/train"
src_val_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset/val"

dst_train_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset Backup/train"
dst_val_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset Backup/val"

print("Begin backing up the 'val' file: ")
image_files = os.listdir(src_val_dir)
for image_file in image_files:
  temp_dir = os.path.join(src_val_dir, image_file)
  shutil.copy(temp_dir, dst_val_dir)
print("The 'val' file has been successfully backed up!")

print("Begin backing up the 'train' file: ")
image_files = os.listdir(src_train_dir)
for image_file in image_files:
  temp_dir = os.path.join(src_train_dir, image_file)
  shutil.copy(temp_dir, dst_train_dir)
print("The 'train' file has been successfully backed up!")



Begin backing up the 'val' file: 
The 'val' file has been successfully backed up!
Begin backing up the 'train' file: 
The 'train' file has been successfully backed up!


## DIODE Dataset Modification
1. depth.npy rescale from 0.6m-350m to 0.6m-10m
2. generate new depth_mask.npy according to new depth.npy
3. generate colorized depth images (depth_hue.png) using Hue as input for Mask R-CNN


In [ ]:
import os
import shutil
import fnmatch
import numpy as np
import cv2
import time
from tqdm import tqdm


def depth2hue(depth_arr):
    d_min = 600
    d_max = 10000
    [rows, cols] = depth_arr.shape
    hue_image = np.zeros([rows, cols, 3])
    for i in range(0, rows):
        for j in range(0, cols):
            d = depth_arr[i, j]
            if d < d_min or d > d_max:
                hue_image[i, j, :] = 0
            else:
                d_normal = 1529 * (d - d_min) / (d_max - d_min)
                # pr
                if (0 <= d_normal <= 255) or (1257 < d_normal <= 1529):
                    hue_image[i, j, 0] = 255
                elif 255 < d_normal <= 510:
                    hue_image[i, j, 0] = 510 - d_normal
                elif 510 < d_normal <= 1020:
                    hue_image[i, j, 0] = 0
                elif 1020 < d_normal <= 1275:
                    hue_image[i, j, 0] = d_normal - 1020
                # pg
                if 0 < d_normal <= 255:
                    hue_image[i, j, 1] = d_normal
                elif 255 < d_normal <= 510:
                    hue_image[i, j, 1] = 255
                elif 510 < d_normal <= 765:
                    hue_image[i, j, 1] = 255
                elif 765 < d_normal <= 1020:
                    hue_image[i, j, 1] = 1020 - d_normal
                elif 1020 < d_normal <= 1529:
                    hue_image[i, j, 1] = 0
                # pb
                if 0 < d_normal <= 510:
                    hue_image[i, j, 2] = 0
                elif 510 < d_normal <= 765:
                    hue_image[i, j, 2] = d_normal - 510
                elif 765 < d_normal <= 1020:
                    hue_image[i, j, 2] = 255
                elif 1020 < d_normal <= 1275:
                    hue_image[i, j, 2] = 255
                elif 1275 < d_normal <= 1529:
                    hue_image[i, j, 2] = 1530 - d_normal
    hue_image = np.array(hue_image, dtype='uint8')
    return hue_image


def depth_normalization(depth_arr):
    d_min = 600
    d_max = 10000
    [rows, cols] = depth_arr.shape
    mask_arr = np.zeros([rows, cols])
    for i in range(0, rows):
        for j in range(0, cols):
            d = depth_arr[i, j]
            if d < d_min or d > d_max:
                depth_arr[i, j] = 0
            else:
              mask_arr[i, j] = 1
    depth_arr = np.array(depth_arr, dtype='uint16')
    return depth_arr, mask_arr


root = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset Backup"
#subsets = ['val', 'train']
subsets = ['train']
namelist = [] # store the basic RGB image (.png) names
full_namelist = [] # store all the file names in the folder 'train' or 'val'
for subset in subsets:
  if subset == 'val':
    total = 771
    print("Begin processing the 'val' folder ")
  elif subset == 'train':
    total = 25458
    print("Begin processing the 'train' folder ")
  basepath = os.path.join(root, subset)
  file_handle1 = open(basepath+'/rgb images namelist.txt',mode='w')
  file_handle2 = open(basepath+'/full library.txt',mode='w')
  file_handle3 = open(basepath+'/files have done.txt',mode='w')
  with os.scandir(basepath) as entries:
    for entry in entries:
      full_namelist.append(entry.name)
      file_handle2.write(entry.name)
      file_handle2.write('\n')
      if entry.name.endswith('.png') and not fnmatch.fnmatch(entry.name, '*_depth_visualization.png'):
        namelist.append(entry.name)
        file_handle1.write(entry.name)
        file_handle1.write('\n')
    file_handle1.close()
    file_handle2.close()
  print("Finish collecting the filenames in the current folder ")
  # Save the lists 'namelist' and 'full_namelist' as files

  print("Namelist has been successfully written into file ")
  print("Full_namelist has been successfully written into file ")
  with tqdm(total=total, desc=subset, leave=True, unit='img', unit_scale=True) as pbar:
    for rgb_name in namelist:
      rgb_name_split_list = rgb_name.split('.')
      temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth.npy')
      depth_arr = np.load(temp_dir) # (768, 1024, 1)
      depth_arr = np.reshape(depth_arr, (768, 1024))
      depth_arr = 1000 * depth_arr
      depth_arr, mask_arr = depth_normalization(depth_arr) # Rescale to 0.6m-10m and return the corresponding mask file
      temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth_new.npy')
      np.save(temp_dir, depth_arr) # Create *_depth_new.npy file

      temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth_mask.npy')
      np.save(temp_dir, mask_arr) # Overwrite *_depth_mask.npy file
      
      depth_hue_arr = depth2hue(depth_arr)
      temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth_visualization.png')
      depth_hue_arr_bgr = cv2.cvtColor(depth_hue_arr, cv2.COLOR_RGB2BGR)
      cv2.imwrite(temp_dir, depth_hue_arr_bgr) # Create the depth visualization file *_depth_visualization.png

      file_handle3.write(rgb_name)
      file_handle3.write('\n')
      pbar.update(1)
    file_handle3.close()
  print("Successfully modified the files in "+ subset +" folder! ")
print("The whole operation has been successfully completed!!! ")
  


In [ ]:
import os
import fnmatch


root = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset Backup"
subsets = ['val']
namelist = [] # store the basic RGB image (.png) names
full_namelist = [] # store all the file names in the folder 'train' or 'val'
for subset in subsets:
  print("Begin processing the 'val' folder ")
  basepath = os.path.join(root, subset)
  file_handle1 = open(basepath+'/rgb images namelist.txt',mode='w')
  file_handle2 = open(basepath+'/full library.txt',mode='w')
  file_handle3 = open(basepath+'/files have done.txt',mode='w')
  file_handle4 = open(basepath+'/val images list.txt',mode='w')
  with os.scandir(basepath) as entries:
    for entry in entries:
      full_namelist.append(entry.name)
      file_handle2.write(entry.name+'\n')
      if entry.name.endswith('.png') and not fnmatch.fnmatch(entry.name, '*_depth_visualization.png'):
        namelist.append(entry.name)
        file_handle1.write(entry.name+'\n')
        file_handle3.write(entry.name+'\n')
        file_handle4.write(entry.name+'\n')
    file_handle1.close()
    file_handle2.close()
    file_handle3.close()
    file_handle4.close()
  print("Finish collecting the filenames in the current folder ")
  # Save the lists 'namelist' and 'full_namelist' as files
  print("Namelist has been successfully written into file ")
  print("Full_namelist has been successfully written into file ")



Begin processing the 'val' folder 
Finish collecting the filenames in the current folder 
Namelist has been successfully written into file 
Full_namelist has been successfully written into file 


## Continue to run DIODE Dataset Modification program for train folder

In [ ]:
import os
import shutil
import fnmatch
import numpy as np
import cv2
import time
from tqdm import tqdm


def depth2hue(depth_arr):
    d_min = 600
    d_max = 10000
    [rows, cols] = depth_arr.shape
    hue_image = np.zeros([rows, cols, 3])
    for i in range(0, rows):
        for j in range(0, cols):
            d = depth_arr[i, j]
            if d < d_min or d > d_max:
                hue_image[i, j, :] = 0
            else:
                d_normal = 1529 * (d - d_min) / (d_max - d_min)
                # pr
                if (0 <= d_normal <= 255) or (1257 < d_normal <= 1529):
                    hue_image[i, j, 0] = 255
                elif 255 < d_normal <= 510:
                    hue_image[i, j, 0] = 510 - d_normal
                elif 510 < d_normal <= 1020:
                    hue_image[i, j, 0] = 0
                elif 1020 < d_normal <= 1275:
                    hue_image[i, j, 0] = d_normal - 1020
                # pg
                if 0 < d_normal <= 255:
                    hue_image[i, j, 1] = d_normal
                elif 255 < d_normal <= 510:
                    hue_image[i, j, 1] = 255
                elif 510 < d_normal <= 765:
                    hue_image[i, j, 1] = 255
                elif 765 < d_normal <= 1020:
                    hue_image[i, j, 1] = 1020 - d_normal
                elif 1020 < d_normal <= 1529:
                    hue_image[i, j, 1] = 0
                # pb
                if 0 < d_normal <= 510:
                    hue_image[i, j, 2] = 0
                elif 510 < d_normal <= 765:
                    hue_image[i, j, 2] = d_normal - 510
                elif 765 < d_normal <= 1020:
                    hue_image[i, j, 2] = 255
                elif 1020 < d_normal <= 1275:
                    hue_image[i, j, 2] = 255
                elif 1275 < d_normal <= 1529:
                    hue_image[i, j, 2] = 1530 - d_normal
    hue_image = np.array(hue_image, dtype='uint8')
    return hue_image


def depth_normalization(depth_arr):
    d_min = 600
    d_max = 10000
    [rows, cols] = depth_arr.shape
    mask_arr = np.zeros([rows, cols])
    for i in range(0, rows):
        for j in range(0, cols):
            d = depth_arr[i, j]
            if d < d_min or d > d_max:
                depth_arr[i, j] = 0
            else:
              mask_arr[i, j] = 1
    depth_arr = np.array(depth_arr, dtype='uint16')
    return depth_arr, mask_arr


root = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset Backup"
#subsets = ['val', 'train']
subsets = ['train']
namelist = [] # store the basic RGB image (.png) names
full_namelist = [] # store all the file names in the folder 'train' or 'val'
left_files = []
done_list = []
for subset in subsets:
  if subset == 'val':
    total = 771
    print("Begin processing the 'val' folder ")
  elif subset == 'train':
    total = 25458
    print("Begin processing the 'train' folder ")
  basepath = os.path.join(root, subset)

  file_handle3 = open(basepath+'/files have done.txt',mode='r')
  contents = file_handle3.readlines()
  for name in contents:
    name = name.strip('\n')
    done_list.append(name)
  file_handle3.close()

  file_handle1 = open(basepath+'/rgb images namelist.txt',mode='r')
  contents = file_handle1.readlines()
  for name in contents:
    name = name.strip('\n')
    namelist.append(name)
  file_handle1.close()

  for file_name in namelist:
    if file_name not in done_list:
      left_files.append(file_name)
  total = total - len(done_list)
  print("Successfully loading the left file names ")
  #file_handle3 = open(basepath+'/files have done.txt',mode='a')
  with open(basepath+'/files have done.txt',mode='a') as file_object:
    with tqdm(total=total, desc=subset, leave=True, unit='img', unit_scale=True) as pbar:
      for rgb_name in left_files:
        rgb_name_split_list = rgb_name.split('.')
        temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth.npy')
        depth_arr = np.load(temp_dir) # (768, 1024, 1)
        depth_arr = np.reshape(depth_arr, (768, 1024))
        depth_arr = 1000 * depth_arr
        depth_arr, mask_arr = depth_normalization(depth_arr) # Rescale to 0.6m-10m and return the corresponding mask file
        temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth_new.npy')
        np.save(temp_dir, depth_arr) # Create *_depth_new.npy file

        temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth_mask.npy')
        np.save(temp_dir, mask_arr) # Overwrite *_depth_mask.npy file
        
        depth_hue_arr = depth2hue(depth_arr)
        temp_dir = os.path.join(basepath, rgb_name_split_list[0]+'_depth_visualization.png')
        depth_hue_arr_bgr = cv2.cvtColor(depth_hue_arr, cv2.COLOR_RGB2BGR)
        cv2.imwrite(temp_dir, depth_hue_arr_bgr) # Create the depth visualization file *_depth_visualization.png

        file_object.write(rgb_name+'\n')
        #file_object.write('\n')
        pbar.update(1)
      #file_handle3.close()
  print("Successfully modified the files in "+ subset +" folder! ")
print("The whole operation has been successfully completed!!! ")

